## Exploratory Data Analysis with SQL and Python

### Introduction

Using this Python notebook you will:

1. Understand the Spacex DataSet
2. Load the dataset into the corresponding table in a Db2 database
3. Execute SQL queries to answer assignment questions

### Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010. SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.

### Connect to the Database

Let us first load the SQL extension and establish a connection with the database

In [1]:
%load_ext sql

In [2]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [3]:
%sql sqlite:///my_data1.db

In [4]:
import pandas as pd
df = pd.read_csv('Spacex.csv')
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False, method='multi')

101

**Note: This below code is added to remove blank rows from table**

In [5]:
%sql create table SPACEXTABLE as select * from SPACEXTBL where Date is not null

 * sqlite:///my_data1.db
Done.


[]

### Tasks

#### 1. Display the Names

Display the names of the unique launch site in the space mission

In [10]:
%sql select distinct launch_site from SPACEXTABLE;

 * sqlite:///my_data1.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


#### 2. Display 5 Records

Display 5 records where launch sites begin with the string 'CCA'.

In [11]:
%sql select * from SPACEXTABLE where launch_site like 'CCA%' limit 5;

 * sqlite:///my_data1.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


#### 3. Display the total payload mass

Display the total payload mass carried by boosters launched by NASA (CRS)

In [12]:
%sql select sum(payload_mass__kg_) as total_payload_mass from SPACEXTABLE where customer = 'NASA (CRS)';

 * sqlite:///my_data1.db
Done.


total_payload_mass
45596


#### 4. Display Average Payload Mass

Display average payload mass carried by booster version F9 v1.1

In [14]:
%sql select avg(payload_mass__kg_) as average_payload_mass from SPACEXTABLE where booster_version like '%F9 v1.1%';

 * sqlite:///my_data1.db
Done.


average_payload_mass
2534.6666666666665


#### 5. List the date

List the date when the first succesful landing outcome in ground pad was acheived.

In [16]:
%sql select min(date) as first_successful_landing from SPACEXTABLE where landing_outcome = 'Success (ground pad)';

 * sqlite:///my_data1.db
Done.


first_successful_landing
2015-12-22


#### 6. List the names of the Boosters

List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [20]:
%sql select booster_version from SPACEXTABLE where landing_outcome = 'Success (drone ship)' and payload_mass__kg_ between 4000 and 6000;

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


#### 7. Number of successful and failure

List the total number of successful and failure mission outcomes

In [21]:
%sql select mission_outcome, count(*) as total_number from SPACEXTABLE group by mission_outcome;

 * sqlite:///my_data1.db
Done.


Mission_Outcome,total_number
Failure (in flight),1
Success,98
Success,1
Success (payload status unclear),1


#### 8. Names of the booster_versions

List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [22]:
%sql select booster_version from SPACEXTABLE where payload_mass__kg_ = (select max(payload_mass__kg_) from SPACEXTABLE);

 * sqlite:///my_data1.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


##### 9. Record which will display the month names

List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use substr(Date, 4, 2) as month to get the months and substr(Date,7,4)='2015' for year.**

In [29]:
%%sql
SELECT strftime('%B', date) AS month, date, booster_version, launch_site, landing_outcome
FROM SPACEXTABLE
WHERE landing_outcome = 'Failure (drone ship)' AND strftime('%Y', date) = '2015';

 * sqlite:///my_data1.db
Done.


month,Date,Booster_Version,Launch_Site,Landing_Outcome
None,2015-01-10,F9 v1.1 B1012,CCAFS LC-40,Failure (drone ship)
None,2015-04-14,F9 v1.1 B1015,CCAFS LC-40,Failure (drone ship)


#### 10. Rank the count of landing outcomes

Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

In [32]:
%%sql select landing_outcome, count(*) as count_outcomes from SPACEXTABLE
      where date between '2010-06-04' and '2017-03-20'
      group by landing_outcome
      order by count_outcomes desc;

 * sqlite:///my_data1.db
Done.


Landing_Outcome,count_outcomes
No attempt,10
Success (drone ship),5
Failure (drone ship),5
Success (ground pad),3
Controlled (ocean),3
Uncontrolled (ocean),2
Failure (parachute),2
Precluded (drone ship),1


**fin.**